In [1]:
import json
import os
import pandas as pd
from datetime import datetime, timedelta
from time import sleep
from itertools import combinations
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By

from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
import warnings
warnings.filterwarnings("ignore")


In [2]:
def init_driver(url):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH,options=chrome_options)
    driver.maximize_window()
    try:
        driver.get(url)
    except:
        print('error 4')
            
    return driver


def wait_for_page_load(xpath_url):
    global driver
    try:
        element_present = EC.presence_of_element_located((By.XPATH, xpath_url))
        WebDriverWait(driver, 5).until(element_present)
        return 1
    except TimeoutException:
        
        print("Timed out waiting for page to load")
        try:
            all_text = driver.find_element_by_xpath('//div[@class="_2jvV"]').text
            if 'No flights found' in all_text:
                return 2
            
            driver.close()
        except:
            driver = init_driver(url)
            print('driver not found')
            
        # sleep(2)
        
        return False
    except NoSuchElementException:
        print("Element not found")
        return False


In [3]:
print('Starting...')
url = 'https://paytm.com/flights'
CHROMEDRIVER_PATH = "C:/Users/swati/Downloads/chromedriver_win32 (1)/chromedriver.exe"
WINDOW_SIZE = "1920,1080"
chrome_options = Options()
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_experimental_option("useAutomationExtension", False)
chrome_options.add_experimental_option("excludeSwitches",["enable-automation"])
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

print('initiated...')

preferences = {
	# "profile.default_content_settings.popups": 0,
	"download.default_directory": os.getcwd() + os.path.sep,
	# "directory_upgrade": True
}
chrome_options.add_experimental_option('prefs', preferences)

# chrome_options.binary_location = CHROME_PATH

driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH, options=chrome_options)
driver.maximize_window()


driver.get(url)


Starting...
initiated...


In [4]:
pd.options.display.max_rows = 100


In [5]:
citys = [
    'DEL-Delhi',
    'BOM-Mumbai',
    'BLR-Bengaluru',
    'HYD-Hyderabad',
    'CU-Kolkata',
    'MAA-Chennai',
    'AMD-Ahmedabad',
    'GOI-Goa',
    'PAT-Patna',
    'COK-Kochi',
    'LKO-Lucknow',
    'GAU-Guwahati',
    'PNQ-Pune',
    'JAI-Jaipur',
    'SXR-Srinagar',
    'BBI-Bhubaneshwar',
    'IXB-Bagdogra',
    'VNS-Varanasi',
    'IXC-Chandigarh',
    'IXR-Ranchi',
    'VTZ-Vishakhapatnam',
    'RPR-Raipur',
    'NAG-Nagpur',
    'TRV-Thiruvananthapuram',
    'CCJ-Kozhikode',
    'IDR-Indore',
    'ATQ-Amritsar',
    'IXJ-Jammu',
    'CJB-Coimbatore',
    'DED-Dehradun',
]


In [6]:
possible_combinations = list(combinations(citys,2))


In [7]:
def collect_data(driver, today, next_date, day_before, source, destination):
    
    status_1 = wait_for_page_load('//div[@class="_2JLq"]/div[@class="_2TFk"]')
    status_2 = wait_for_page_load('//div[@class="_3H-S _1Eia"]')
    
    if status_1 == 2 and status_2 == 2:
        print('flight not found')
        return
    
    all_boxes = driver.find_elements_by_xpath('//div[@class="_2JLq"]/div[@class="_2TFk"]')
    try:
        for one_box in all_boxes:
            try:
                flight_name = one_box.find_element_by_xpath('.//div[@class="_3H-S _1Eia"]').text
                flight_id = one_box.find_element_by_xpath('.//div[@class="NqXj _2GoO"]').text
                start_time = one_box.find_element_by_xpath('.//div[@class="_3H-S"]').text
                end_time = one_box.find_element_by_xpath('.//div[@class="_3H-S _1wD5"]').text.split('\n')[0]
                travel_time = one_box.find_element_by_xpath('.//div[@class="vY4t"]').text
                stops = one_box.find_element_by_xpath('.//div[@class="_7BOG"]').text
                fare = one_box.find_element_by_xpath('.//div[@class="_1cxG"]').text
                 
                all_data.append({
                    'flight_name': flight_name,
                    'flight_id': flight_id,
                    'start_time': start_time,
                    'end_time': end_time,
                    'travel_time': travel_time,
                    'stops': stops,
                    'fare': fare,
                    'source': source,
                    'destination': destination
                })
            except Exception as e:
                print(e)
                print('error2')
                # sleep(5)
            
    except Exception as e:
        print(e)
        print('error3')
        # sleep(5)


In [8]:
def search_page(driver, source, destination):
    
    try:
        clear_inputs = driver.find_elements_by_xpath('//div[@class="fl-input-clear"]')
        clear_inputs[0].click()
        clear_inputs[1].click()
    except:
        print('cleared already')
        
    sleep(2)
    wait_for_page_load('//div[@class="_31PN"]//input')
    
    inputs = driver.find_elements_by_xpath('//div[@class="_31PN"]//input')

    start_input = inputs[0]
    end_input = inputs[1]

    start_input.clear()
    start_input.send_keys(source)
    sleep(2)
    wait_for_page_load('//div[@class="_2Alu"]//div[@class="_2xgL"]')
    # Select First suggestion
    driver.find_element_by_xpath('//div[@class="_2Alu"]//div[@class="_2xgL"]').click()

    end_input.clear()
    end_input.send_keys(destination)
    sleep(2)
    
    wait_for_page_load('//div[@class="_2Alu"]//div[@class="_2xgL"]')
    # Select First suggestion
    driver.find_element_by_xpath('//div[@class="_2Alu"]//div[@class="_2xgL"]').click()

    driver.find_element_by_xpath("//button[text()='Search']").click()
    sleep(2)


In [11]:
all_data = []
for i, one_combination in enumerate(possible_combinations):
    source, destination = one_combination
    today = datetime.today() + timedelta(5)
    print(today)
    try:
        next_date = today.strftime('%Y-%m-%d') # (today + timedelta(i)).strftime('%Y-%m-%d')
        print(next_date)
        search_page(driver, source, destination)
        sleep(2)
        collect_data(driver, today, next_date, 0, source, destination)
    except Exception as e:
        # sleep(2)
        print(e)
        print('error1')
        try:
            driver.close()
        except:
            print('driver not found')
            pd.DataFrame(all_data).to_csv('flight_data__.csv')
            
        driver = init_driver(url)
        
    print(one_combination, i+1, len(possible_combinations), (i+1)/ len(possible_combinations))
df = pd.DataFrame(all_data)
df


2022-03-26 08:02:21.040551
2022-03-26
cleared already
Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=99.0.4844.51)
Stacktrace:
Backtrace:
	Ordinal0 [0x007E7AC3+2587331]
	Ordinal0 [0x0077ADD1+2141649]
	Ordinal0 [0x00673BB8+1063864]
	Ordinal0 [0x0065C5D0+968144]
	Ordinal0 [0x006BAE49+1355337]
	Ordinal0 [0x006C8192+1409426]
	Ordinal0 [0x006B84A6+1344678]
	Ordinal0 [0x006953F6+1201142]
	Ordinal0 [0x006962E6+1204966]
	GetHandleVerifier [0x0098DF22+1680738]
	GetHandleVerifier [0x00A40DBC+2413564]
	GetHandleVerifier [0x0087D151+563089]
	GetHandleVerifier [0x0087BF13+558419]
	Ordinal0 [0x0078081E+2164766]
	Ordinal0 [0x00785508+2184456]
	Ordinal0 [0x00785650+2184784]
	Ordinal0 [0x0078F5BC+2225596]
	BaseThreadInitThunk [0x76946739+25]
	RtlGetFullPathName_UEx [0x77DF8E7F+1215]
	RtlGetFullPathName_UEx [0x77DF8E4D+1165]

error1
driver not found
('DEL-Delhi', 'BOM-Mumbai') 1 435 0.0022988505747126436
2022-03-26 08:02:36.975195
202

('BLR-Bengaluru', 'NAG-Nagpur') 77 435 0.17701149425287357
2022-03-26 08:31:51.955585
2022-03-26
('BLR-Bengaluru', 'TRV-Thiruvananthapuram') 78 435 0.1793103448275862
2022-03-26 08:32:07.434604
2022-03-26
('BLR-Bengaluru', 'CCJ-Kozhikode') 79 435 0.18160919540229886
2022-03-26 08:32:18.855321
2022-03-26
('BLR-Bengaluru', 'IDR-Indore') 80 435 0.1839080459770115
2022-03-26 08:32:35.575877
2022-03-26
('BLR-Bengaluru', 'ATQ-Amritsar') 81 435 0.18620689655172415
2022-03-26 08:32:53.874126
2022-03-26
('BLR-Bengaluru', 'IXJ-Jammu') 82 435 0.18850574712643678
2022-03-26 08:33:12.607905
2022-03-26
('BLR-Bengaluru', 'CJB-Coimbatore') 83 435 0.19080459770114944
2022-03-26 08:33:26.434151
2022-03-26
('BLR-Bengaluru', 'DED-Dehradun') 84 435 0.19310344827586207
2022-03-26 08:33:43.975286
2022-03-26
('HYD-Hyderabad', 'CU-Kolkata') 85 435 0.19540229885057472
2022-03-26 08:34:15.651757
2022-03-26
('HYD-Hyderabad', 'MAA-Chennai') 86 435 0.19770114942528735
2022-03-26 08:34:36.908986
2022-03-26
('HYD-Hyd

('MAA-Chennai', 'RPR-Raipur') 151 435 0.3471264367816092
2022-03-26 08:52:23.934059
2022-03-26
('MAA-Chennai', 'NAG-Nagpur') 152 435 0.34942528735632183
2022-03-26 08:52:36.363480
2022-03-26
('MAA-Chennai', 'TRV-Thiruvananthapuram') 153 435 0.35172413793103446
2022-03-26 08:52:49.136517
2022-03-26
('MAA-Chennai', 'CCJ-Kozhikode') 154 435 0.35402298850574715
2022-03-26 08:53:00.424619
2022-03-26
('MAA-Chennai', 'IDR-Indore') 155 435 0.3563218390804598
2022-03-26 08:53:14.899937
2022-03-26
('MAA-Chennai', 'ATQ-Amritsar') 156 435 0.3586206896551724
2022-03-26 08:53:28.486513
2022-03-26
('MAA-Chennai', 'IXJ-Jammu') 157 435 0.36091954022988504
2022-03-26 08:53:40.749636
2022-03-26
('MAA-Chennai', 'CJB-Coimbatore') 158 435 0.3632183908045977
2022-03-26 08:53:52.503384
2022-03-26
('MAA-Chennai', 'DED-Dehradun') 159 435 0.36551724137931035
2022-03-26 08:54:07.257324
2022-03-26
('AMD-Ahmedabad', 'GOI-Goa') 160 435 0.367816091954023
2022-03-26 08:54:24.303840
2022-03-26
('AMD-Ahmedabad', 'PAT-Pa

('COK-Kochi', 'LKO-Lucknow') 226 435 0.5195402298850574
2022-03-26 09:09:05.692970
2022-03-26
('COK-Kochi', 'GAU-Guwahati') 227 435 0.5218390804597701
2022-03-26 09:09:18.297628
2022-03-26
('COK-Kochi', 'PNQ-Pune') 228 435 0.5241379310344828
2022-03-26 09:09:30.900814
2022-03-26
('COK-Kochi', 'JAI-Jaipur') 229 435 0.5264367816091954
2022-03-26 09:09:45.024095
2022-03-26
('COK-Kochi', 'SXR-Srinagar') 230 435 0.5287356321839081
2022-03-26 09:09:56.329517
2022-03-26
('COK-Kochi', 'BBI-Bhubaneshwar') 231 435 0.5310344827586206
2022-03-26 09:10:09.006500
2022-03-26
('COK-Kochi', 'IXB-Bagdogra') 232 435 0.5333333333333333
2022-03-26 09:10:22.358489
2022-03-26
('COK-Kochi', 'VNS-Varanasi') 233 435 0.535632183908046
2022-03-26 09:10:34.967777
2022-03-26
('COK-Kochi', 'IXC-Chandigarh') 234 435 0.5379310344827586
2022-03-26 09:10:47.394165
2022-03-26
('COK-Kochi', 'IXR-Ranchi') 235 435 0.5402298850574713
2022-03-26 09:10:59.048308
2022-03-26
('COK-Kochi', 'VTZ-Vishakhapatnam') 236 435 0.54252873

('JAI-Jaipur', 'IDR-Indore') 311 435 0.7149425287356321
2022-03-26 09:27:19.858724
2022-03-26
('JAI-Jaipur', 'ATQ-Amritsar') 312 435 0.7172413793103448
2022-03-26 09:27:34.309655
2022-03-26
('JAI-Jaipur', 'IXJ-Jammu') 313 435 0.7195402298850575
2022-03-26 09:27:46.613063
2022-03-26
('JAI-Jaipur', 'CJB-Coimbatore') 314 435 0.7218390804597701
2022-03-26 09:27:59.517467
2022-03-26
('JAI-Jaipur', 'DED-Dehradun') 315 435 0.7241379310344828
2022-03-26 09:28:12.458427
2022-03-26
('SXR-Srinagar', 'BBI-Bhubaneshwar') 316 435 0.7264367816091954
2022-03-26 09:28:26.492720
2022-03-26
('SXR-Srinagar', 'IXB-Bagdogra') 317 435 0.728735632183908
2022-03-26 09:28:39.084972
2022-03-26
('SXR-Srinagar', 'VNS-Varanasi') 318 435 0.7310344827586207
2022-03-26 09:28:53.609305
2022-03-26
('SXR-Srinagar', 'IXC-Chandigarh') 319 435 0.7333333333333333
2022-03-26 09:29:07.836899
2022-03-26
('SXR-Srinagar', 'IXR-Ranchi') 320 435 0.735632183908046
2022-03-26 09:29:20.950795
2022-03-26
('SXR-Srinagar', 'VTZ-Vishakhap

('VTZ-Vishakhapatnam', 'TRV-Thiruvananthapuram') 393 435 0.903448275862069
2022-03-26 10:14:53.199578
2022-03-26
('VTZ-Vishakhapatnam', 'CCJ-Kozhikode') 394 435 0.9057471264367816
2022-03-26 10:15:04.694507
2022-03-26
('VTZ-Vishakhapatnam', 'IDR-Indore') 395 435 0.9080459770114943
2022-03-26 10:15:17.633055
2022-03-26
('VTZ-Vishakhapatnam', 'ATQ-Amritsar') 396 435 0.9103448275862069
2022-03-26 10:15:30.950948
2022-03-26
('VTZ-Vishakhapatnam', 'IXJ-Jammu') 397 435 0.9126436781609195
2022-03-26 10:15:42.588784
2022-03-26
('VTZ-Vishakhapatnam', 'CJB-Coimbatore') 398 435 0.9149425287356322
2022-03-26 10:15:56.210742
2022-03-26
('VTZ-Vishakhapatnam', 'DED-Dehradun') 399 435 0.9172413793103448
2022-03-26 10:16:08.838922
2022-03-26
('RPR-Raipur', 'NAG-Nagpur') 400 435 0.9195402298850575
2022-03-26 10:16:20.857739
2022-03-26
('RPR-Raipur', 'TRV-Thiruvananthapuram') 401 435 0.9218390804597701
2022-03-26 10:16:33.562510
2022-03-26
('RPR-Raipur', 'CCJ-Kozhikode') 402 435 0.9241379310344827
2022-0

,flight_name,flight_id,start_time,end_time,travel_time,stops,fare,source,destination
0,SpiceJet,SG - 534,22:30,01:15,2h 45m,Non Stop,"8,160",DEL-Delhi,BLR-Bengaluru
1,IndiGo,6E - 2036,22:50,01:45,2h 55m,Non Stop,"8,160",DEL-Delhi,BLR-Bengaluru
2,Go First,G8 - 275,21:45,03:00,5h 15m,1 stop at Pune,"8,160",DEL-Delhi,BLR-Bengaluru
3,Go First,G8 - 2511,10:45,16:05,5h 20m,1 stop at Patna,"8,160",DEL-Delhi,BLR-Bengaluru
4,Go First,G8 - 165,10:00,16:05,6h 5m,1 stop at Patna,"8,160",DEL-Delhi,BLR-Bengaluru
...,...,...,...,...,...,...,...,...,...
7922,Air India,AI - 539,15:05,13:45,22h 40m,"2 Stops (MAA, DEL)","19,497",CJB-Coimbatore,DED-Dehradun
7923,Air India,AI - 679,14:00,07:20,17h 20m,"2 Stops (BOM, DEL)","24,484",CJB-Coimbatore,DED-Dehradun
7924,Air India,AI - 679,14:00,07:20,17h 20m,"2 Stops (BOM, DEL)","24,484",CJB-Coimbatore,DED-Dehradun
7925,Air India,AI - 539,15:05,13:45,22h 40m,"2 Stops (MAA, DEL)","24,642",CJB-Coimbatore,DED-Dehradun


In [12]:
df.to_csv('flight_data_paytm.csv')
